In [ ]:
%pylab inline
import pandas as pd
import numpy as np
#Set thousands separator
pd.set_option('display.float_format', lambda x: '{:,.2f}'.format(x))
#Import my functions
import fin_functions as funcs

In [ ]:
#Set the columns names for exoda
exnames = ['logar','perigrafi','proteinomenos','eggekrimenos','anamorfoseis','diamorfomenos','desmefthenta','entalthenta','proplirothenta','plirothenta']
#Set the columns names for esoda
esnames = ['logar','perigrafi','proteinomenos','eggekrimenos','anamorfoseis','diamorfomenos']

In [ ]:
#Set an array for the filenames
ex_names = ['ex_2012','ex_2013','ex_2014']
es_names = ['es_2012','es_2013','es_2014']

In [ ]:
exdfs = {}
for dfname in ex_names: 
    #print dfname
    exdfs[dfname] = pd.read_excel(dfname +'.xls',sheetname=0,header=None,skiprows=1, parse_cols='A:J',names=exnames)
    
esdfs = {}
for dfname in es_names:
    #print dfname
    esdfs[dfname] = pd.read_excel(dfname +'.xls',sheetname=0,header=None,skiprows=1, parse_cols='A:F',names=esnames)

###Δημιουργία των απαιτούμενων dataframes για ανάλυση με βάση το φορέα

In [ ]:
reload(funcs)
ex_foreas_13 = funcs.build_dataframe_forea(exdfs['ex_2013']) 
es_foreas_13 = funcs.build_dataframe_forea(esdfs['es_2013'])

ex_foreas_14 = funcs.build_dataframe_forea(exdfs['ex_2014']) 
es_foreas_14 = funcs.build_dataframe_forea(esdfs['es_2014'])

### Ενοποίηση του 2013 με βάση την ενότητα_φορέα και ανάλυση.

In [ ]:
merged = pd.merge(es_foreas_13,ex_foreas_13,on='foreas_enotita')
reload(funcs)
merged = funcs.calculate_diffs_on_year(merged)
merged.columns

#### Δημιουργία μερικών γραφημάτων για το 2013

In [ ]:
pivot_enotita = merged.pivot_table('plirothenta',index='enotita_y',columns='foreas_y',aggfunc=sum)
normed_pivot_enotita = pivot_enotita.div(pivot_enotita.sum(1),axis=0)
from matplotlib import cm
cmap = cm.get_cmap('spectral')
normed_pivot_enotita.plot(kind='bar',stacked=True, figsize=(12,8), colormap=cmap)

In [ ]:
normed_pivot_enotita

In [ ]:
filter_merged = merged[merged.foreas_y == '071'].copy()
filter_merged[['foreas_y','perigrafi_y','enotita_y','plirothenta']]

In [ ]:
pivot_forea = merged.pivot_table('plirothenta',index='foreas_y',columns='enotita_y',aggfunc=sum)
normed_pivot_forea = pivot_forea.div(pivot_forea.sum(1),axis=0)
normed_pivot_forea.plot(kind='bar',stacked=True, figsize=(12,8), colormap=cmap)

In [ ]:
normed_pivot_forea

In [ ]:
filter_merged = merged[merged.foreas_y == '293'].copy()
filter_merged[['foreas_y','perigrafi_y','enotita_y','plirothenta','entalthenta','desmefthenta']]

#Καλό είναι να δωθεί ιδιαίτερη προσοχή στους φορείς 721, 722, 723

### Ενοποίηση του 2014 με βάση την ενότητα_φορέα και ανάλυση.

In [ ]:
merged = pd.merge(es_foreas_14,ex_foreas_14,on='foreas_enotita')
reload(funcs)
merged = funcs.calculate_diffs_on_year(merged)
merged.columns

#### Δημιουργία μερικών γραφημάτων για το 2014

In [ ]:
pivot_enotita = merged.pivot_table('plirothenta',index='enotita_y',columns='foreas_y',aggfunc=sum)
normed_pivot_enotita = pivot_enotita.div(pivot_enotita.sum(1),axis=0)
from matplotlib import cm
cmap = cm.get_cmap('spectral')
normed_pivot_enotita.plot(kind='bar',stacked=True, figsize=(12,8), colormap=cmap)

In [ ]:
normed_pivot_enotita

In [ ]:
filter_merged = merged[merged.foreas_y == '071'].copy()
filter_merged[['foreas_y','perigrafi_y','enotita_y','plirothenta']]

In [ ]:
pivot_forea = merged.pivot_table('plirothenta',index='foreas_y',columns='enotita_y',aggfunc=sum)
normed_pivot_forea = pivot_forea.div(pivot_forea.sum(1),axis=0)
normed_pivot_forea.plot(kind='bar',stacked=True, figsize=(12,8), colormap=cmap)

In [ ]:
normed_pivot_forea

In [ ]:
filter_merged = merged[merged.foreas_y == '293'].copy()
filter_merged[['foreas_y','perigrafi_y','enotita_y','plirothenta','entalthenta','desmefthenta']]

##Πλήρης Ενοποίηση

In [ ]:
def make_g(df):
    diam = df[['foreas','diamorfomenos','desmefthenta','entalthenta','plirothenta']]
    grp = diam.groupby('foreas') 
    g = grp.agg({'foreas': lambda x: x.iloc[0],'diamorfomenos':"sum",'desmefthenta':"sum",'entalthenta':"sum",'plirothenta':"sum"})
    return g

def make_g2(df):
    diam = df[['enotita','foreas','diamorfomenos','desmefthenta','entalthenta','plirothenta']]
    grp = diam.groupby(['foreas','enotita'])
    return grp

g13 = make_g(ex_foreas_13)
g14 = make_g(ex_foreas_14)

gg13 = make_g2(ex_foreas_13)
gg14 = make_g2(ex_foreas_14)

In [ ]:
g13.sum()

In [ ]:
g14.sum()

In [ ]:
gm = pd.merge(g13,g14,on="foreas")
gm[['desmefthenta_x','desmefthenta_y']].plot(kind='bar')

In [ ]:
df = exdfs['ex_2014']
df[(df.logar.str[11:]=='000') & (df.logar.str[3:5]=='01') & (df.diamorfomenos != 0)]

In [ ]:
reload(funcs)
df = funcs.format_dataframe(df)

In [ ]:
df[(df.kae == '9000') & (df.foreas == '721')]

In [ ]:
#Γράφημα χιλιάδας
target_var = 'eggekrimenos'
df1 = df[df.last_three == '000'].iloc[:,2:]
df2 = df1.pivot_table(target_var,index='enotita',columns='kae',aggfunc='sum')
df3 = df2.div(df2.sum(1),axis=0)
df3.plot(kind='bar',stacked=True,colormap=cmap,figsize=(12,8))


In [ ]:
#Ανάλυση χιλιάδων σε εκατοντάδες - Μόνο το 0000 που είναι και ο κύριος όγκος
titles={'0':'PLIROMES GIA YPIRESIES',
        '1':'PROMITHIES AGATHWN KAI KEFALAIAKOY EXOPLISMOU',
        '2':'PLIROMES METABIBASTIKES',
        '3':'PLIROMES POU ANTIKRIZONTAI APO PRAGMATOPOIOUMENA ESODA',
        '4':'DAPANES NATO APO KRATOI MELH',
        '5':'DAPANES POU DEN ENTASSONTAI SE ALLES KATIGORIES',
        '6':'PLIROMES GIA THN EXYPHRETHSH THS DHMOSIAS PISTIS',
        '7':'APALOTRIOSEIS, AGORES, ANEGERSEIS ...',
        '8':'PLIROMES GIA ERGA P.D.E',
        '9':'PLIROMES GIA EPENDISEIS'}
for i in '0':
    df1 = df[(df.xiliada == i) & (df.last_two == '00') & (df.last_three != '000')].iloc[:,2:]
    df2 = df1.pivot_table('plirothenta',index='enotita',columns='kae',aggfunc='sum')
    df3 = df2.div(df2.sum(1),axis=0)
    df3.plot(kind='bar',stacked=True,colormap=cmap,figsize=(12,8),title=titles[i])

In [ ]:
#Ανάλυση χιλιάδων σε εκατοντάδες - Οι υπόλοιπες χωρίς τη 0000
for i in '123456789':
    df1 = df[(df.xiliada == i) & (df.last_two == '00') & (df.last_three != '000')].iloc[:,2:]
    df2 = df1.pivot_table('plirothenta',index='enotita',columns='kae',aggfunc='sum')
    df3 = df2.div(df2.sum(1),axis=0)
    df3.plot(kind='bar',stacked=True,colormap=cmap,figsize=(12,8),title=titles[i])

In [ ]:
#Καταγραφή των λογαριασμών που αντιστοιχούν σε φορείς
foreis = df[(df.logar.str.len() == 9) & (df.enotita == '01')]
foreis = foreis[['foreas','perigrafi']]
foreis

In [ ]:
#Ανάλυση σε δεκάδες
temp = df[(df.xiliada == '0') & (df.logar.str[-3:-2] == '8') & (df.logar.str[-2:-1] != '0')  ]
t = df[(df.kae == '0873') & (df.eggekrimenos != 0)]
t[['logar','perigrafi','eggekrimenos']]

In [ ]:
temp